<a href="https://colab.research.google.com/github/styxx216/zoomcamp_mlops/blob/main/zoomcamp_mlops_homework_6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [208]:
!pip install xgboost

In [209]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

from sklearn.metrics import mean_squared_error

from sklearn.tree import DecisionTreeRegressor
from sklearn.feature_extraction import DictVectorizer
from sklearn.metrics import roc_auc_score
from sklearn.tree import export_text

from sklearn.ensemble import RandomForestRegressor

import xgboost as xgb

In [210]:
!wget https://raw.githubusercontent.com/alexeygrigorev/datasets/master/housing.csv


--2023-10-22 22:57:50--  https://raw.githubusercontent.com/alexeygrigorev/datasets/master/housing.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1423529 (1.4M) [text/plain]
Saving to: ‘housing.csv.7’

housing.csv.7       100%[===================>]   1.36M  --.-KB/s    in 0.04s   

2023-10-22 22:57:50 (37.1 MB/s) - ‘housing.csv.7’ saved [1423529/1423529]



In [211]:
housing_df = pd.read_csv('housing.csv')

In [212]:
housing_df.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,NEAR BAY
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,NEAR BAY
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,NEAR BAY
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,NEAR BAY
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,NEAR BAY


In [213]:
housing_df = housing_df.loc[(housing_df['ocean_proximity'] == 'INLAND') | (housing_df['ocean_proximity'] == '<1H OCEAN')]

In [214]:
housing_df['ocean_proximity'] = housing_df['ocean_proximity'].replace('<1H OCEAN','1H OCEAN')

In [215]:
housing_df.isna().sum()

longitude               0
latitude                0
housing_median_age      0
total_rooms             0
total_bedrooms        157
population              0
households              0
median_income           0
median_house_value      0
ocean_proximity         0
dtype: int64

In [216]:
housing_df = housing_df.fillna(0)

In [217]:
housing_df['median_house_value'] = np.log(housing_df['median_house_value'])

In [218]:
df_full_train, df_test = train_test_split(housing_df,test_size = 0.2, random_state = 1)
df_train, df_val = train_test_split(df_full_train, test_size = 0.25, random_state = 1)

In [219]:
y_train = (df_train.median_house_value).values
y_val = (df_val.median_house_value).values
y_test = (df_test.median_house_value).values

In [220]:
del df_train['median_house_value']
del df_val['median_house_value']
del df_test['median_house_value']

1

In [221]:
train_dicts = df_train.to_dict(orient='records')
val_dicts = df_val.to_dict(orient='records')
test_dicts = df_test.to_dict(orient='records')

In [222]:
dv = DictVectorizer(sparse=False)
X_train = dv.fit_transform(train_dicts)
X_val = dv.fit_transform(val_dicts)
X_test = dv.fit_transform(test_dicts)

In [223]:
list(dv.get_feature_names_out())

['households',
 'housing_median_age',
 'latitude',
 'longitude',
 'median_income',
 'ocean_proximity=1H OCEAN',
 'ocean_proximity=INLAND',
 'population',
 'total_bedrooms',
 'total_rooms']

In [224]:
dtr = DecisionTreeRegressor(max_depth=1)
dtr.fit(X_train, y_train)

DecisionTreeRegressor(max_depth=1)

In [225]:
print(export_text(dtr, feature_names=list(dv.get_feature_names_out())))

|--- ocean_proximity=INLAND <= 0.50
|   |--- value: [12.30]
|--- ocean_proximity=INLAND >  0.50
|   |--- value: [11.61]



2

In [226]:
rfr = RandomForestRegressor(n_estimators=10, random_state=1, n_jobs=-1)
rfr.fit(X_train, y_train)

RandomForestRegressor(n_estimators=10, n_jobs=-1, random_state=1)

In [227]:
y_pred = rfr.predict(X_val)
print(round(mean_squared_error(y_pred,y_val)**(1/2),3))

0.245


3

In [228]:
n_estimators = np.arange(10,210,10)

best_n = 0
best_rmse = np.inf
for n in n_estimators:
    rfr = RandomForestRegressor(n_estimators=n, random_state=1, n_jobs=-1)
    rfr.fit(X_train, y_train)
    y_pred = rfr.predict(X_val)
    rmse = mean_squared_error(y_pred,y_val)**(1/2)
    print(f'n_estimators = {n} RMSE = {round(rmse,3)}')
    if rmse<best_rmse:
        best_rmse = rmse
        best_n = n

print(f'best n_estimators = {best_n}')

n_estimators = 10 RMSE = 0.245
n_estimators = 20 RMSE = 0.239
n_estimators = 30 RMSE = 0.237
n_estimators = 40 RMSE = 0.235
n_estimators = 50 RMSE = 0.235
n_estimators = 60 RMSE = 0.235
n_estimators = 70 RMSE = 0.235
n_estimators = 80 RMSE = 0.235
n_estimators = 90 RMSE = 0.234
n_estimators = 100 RMSE = 0.234
n_estimators = 110 RMSE = 0.234
n_estimators = 120 RMSE = 0.234
n_estimators = 130 RMSE = 0.234
n_estimators = 140 RMSE = 0.234
n_estimators = 150 RMSE = 0.234
n_estimators = 160 RMSE = 0.233
n_estimators = 170 RMSE = 0.233
n_estimators = 180 RMSE = 0.234
n_estimators = 190 RMSE = 0.234
n_estimators = 200 RMSE = 0.234
best n_estimators = 170


4

In [229]:
for max_depth in [10, 15, 20, 25]:
    mean_rmse = 0
    for n in n_estimators:
        rfr = RandomForestRegressor(n_estimators=n, max_depth = max_depth, random_state=1, n_jobs=-1)
        rfr.fit(X_train, y_train)
        y_pred = rfr.predict(X_val)
        mean_rmse += mean_squared_error(y_pred,y_val)**(1/2)
    mean_rmse %= len(n_estimators)
    print(f'max_depth = {max_depth} mean RMSE = {round(mean_rmse,3)}')


max_depth = 10 mean RMSE = 4.909
max_depth = 15 mean RMSE = 4.714
max_depth = 20 mean RMSE = 4.691
max_depth = 25 mean RMSE = 4.702


5

In [230]:
rfr = RandomForestRegressor(n_estimators=10, max_depth = 20, random_state=1, n_jobs=-1)
rfr.fit(X_train, y_train)
d = dict()
for i in range(len(list(dv.get_feature_names_out()))):
    d[dv.get_feature_names_out()[i]] = rfr.feature_importances_[i]
def value_getter(item):
    return item[1]
print(sorted(d.items(), key=value_getter, reverse = True))

[('median_income', 0.33574968032761554), ('ocean_proximity=INLAND', 0.33098243681580797), ('latitude', 0.10151452760305246), ('longitude', 0.08709333928795938), ('ocean_proximity=1H OCEAN', 0.035297113178256095), ('housing_median_age', 0.030323451392227505), ('population', 0.028345150071234507), ('total_rooms', 0.02098439729588686), ('total_bedrooms', 0.015154470388830506), ('households', 0.014555433639129103)]


6

In [231]:
features = list(dv.get_feature_names_out())
dtrain = xgb.DMatrix(X_train, label=y_train, feature_names=features)
dval = xgb.DMatrix(X_val, label=y_val, feature_names=features)

In [232]:
watchlist = [(dtrain, 'train'), (dval, 'val')]

In [233]:
xgb_params = {
    'eta': 0.3,
    'max_depth': 6,
    'min_child_weight': 1,

    'objective': 'reg:squarederror',
    'nthread': 8,

    'seed': 1,
    'verbosity': 1,
}
model = xgb.train(xgb_params, dtrain, num_boost_round=100)
y_pred = model.predict(dval)
rmse = mean_squared_error(y_pred,y_val)**(1/2)

print(round(rmse,3))

0.229


In [234]:
xgb_params = {
    'eta': 0.1,
    'max_depth': 6,
    'min_child_weight': 1,

    'objective': 'reg:squarederror',
    'nthread': 8,

    'seed': 1,
    'verbosity': 1,
}
model = xgb.train(xgb_params, dtrain, num_boost_round=100)
y_pred = model.predict(dval)
rmse = mean_squared_error(y_pred,y_val)**(1/2)

print(round(rmse,3))

0.232
